<a href="https://colab.research.google.com/github/Marcelo-Ferraz-de-Oliveira/Controle-aplicacoes-financeiras/blob/main/Extrator_nota_corretagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Extrator de informações das operações descritas na nota de corretagem
!pip install tabula-py
!pip install pandas
import tabula
import pandas as pd
import json


In [ ]:
def str_to_br_currency(string):
  #print(string)
  string = string[::-1]
  string = string.replace('.','')
  string = string.replace(',','.',1)
  string = string.replace(',','')
  string = string[::-1]
  #print(string)
  return float(string)

#tests
#display(str_to_br_currency('1.432,00'))
# display(str_to_br_currency('600,00'))
# display(str_to_br_currency('500,32'))
# display(str_to_br_currency('1,432,32'))

def us_currency_to_float(n):
  #number with 2 or more dots (pandas will convert to string)
  #print(n, int(n), n == int(n))
  if type(n) == str:
    return float(n.replace('.',''))
  #number with 1 dot
  elif n == int(n):
    return float(n)
  else:
    #print(float(n*1000)) 
    return float(n*1000)

#tests
# display(us_currency_to_float(100.0))
#display(us_currency_to_float(1.8))
# display(us_currency_to_float('1.100.0'))
# display(us_currency_to_float('1.000.000'))


def fix_sep_negocios(df):
  word_df = df.copy()
  #display(df)
  word_df['Preço'] = word_df['Preço'].apply(str_to_br_currency)
  word_df['Valor Operação'] = word_df['Valor Operação'].apply(str_to_br_currency)
  word_df['Quantidade'] = word_df['Quantidade'].apply(us_currency_to_float)
  #display(df)
  return word_df

def fix_sep_custos(df):
  word_df = df.copy()
  #display(word_df['Valor'])
  word_df['Valor'] = word_df['Valor'].apply(str_to_br_currency)
  #display(df)
  return word_df

In [62]:
def definir_corretora(file_to_open, passwd):
  #corretora = ''
  area_header = [0,0,28,100]
  df_corretora = tabula.io.read_pdf(file_to_open, pages = 1, area=area_header, stream = True, relative_area= True, password = passwd, output_format='json')
  df_corretora = json.dumps(df_corretora)
  if (df_corretora.find('clear') != -1):
    return 'clear'
  if (df_corretora.find('genial') != -1):
    return 'genial'
  #elif (df_corretora.find('xp') != -1):
  #  corretora = 'xp'
  return ''

#tests
#definir_corretora(file_to_open,passwd)

In [66]:
def integrate(datas, negocios, custos):
  if len(datas) != len(negocios): return []
  var = []
  for n in range(0,len(datas)):
    var.append([datas[n],negocios[n], custos[n]])
  return var

def extrair_data_clear(file_to_open, passwd, formato = 'dataframe'):
  #Date and note number extraction
  area_datas = [0,70,10,100]
  df_datas = tabula.io.read_pdf(file_to_open, stream = True, area=area_datas, pages = 'all',relative_area= True, password = passwd)
  df_datas_c = []
  #display(df_datas)
  for data in df_datas:
   data = data[['Nr. nota','Unnamed: 0','Data pregão']]
   data.columns = ['Nr. nota','Folha','Data']
   if formato == 'json': data = data.to_dict(orient='records')
   df_datas_c.append(data)
#  for data in df_datas_c:
#    display(data)
  return df_datas_c

def extrair_negocios_clear(file_to_open, passwd, formato = 'dataframe'):
    #Trades extraction
    #Portion of the page to analyze(top,left,bottom,right).
    #Take only the trades table
    area_negocios = [28,0,53,100]
    df_negocios = tabula.io.read_pdf(file_to_open, stream = True, area=area_negocios, pages = 'all',relative_area= True, password = passwd)
    #Exclude wrong associations (NaN content), rename columns names and fix numbers separators
    df_negocios_c = []
    for content in df_negocios:
      content = content.dropna(axis=1, how='any')
      if content['Tipo mercado'][0] == 'OPCAO DE COMPRA':
        content = content.drop(['Prazo','Unnamed: 0'], axis = 1)
        content.columns = ['Negociação','C/V','Tipo de mercado','Papel','Quantidade','Preço','Valor Operação','D/C']
      elif content['Tipo mercado'][0] == 'VISTA':
        content.columns = ['Negociação','C/V','Tipo de mercado','Papel','Quantidade','Preço','Valor Operação','D/C']
      else: continue
      content = fix_sep_negocios(content)
      if formato == 'json': content = content.to_dict(orient='records')
      df_negocios_c.append(content)
    return df_negocios_c

def extrair_custos_clear(file_to_open, passwd, formato='dataframe'):
    #Costs extraction
    area_custos = [53,50,95,100]
    df_custos = tabula.io.read_pdf(file_to_open, stream = True, area=area_custos, pages = 'all',relative_area= True, password = passwd, columns=[450,550,600])
    df_custos_c = []
    for content in df_custos:
      content = pd.concat((content[2:4],content[6:9],content[12:18]))
      content.columns=['Custo','Valor','C/D']
      content['Custo'][16] = 'IRRF'
      content['Valor'] = content['Valor'].fillna('0')
      content['C/D'] = content['C/D'].fillna('N/A')
      content = fix_sep_custos(content)
      #content = content.set_index('Custo')
      if formato =='json': content = content.to_dict(orient='records')
      df_custos_c.append(content)
    return df_custos_c

#tests
#extrair_data_clear(files[0],passwd)


def extrair_dados(file_to_open, passwd, formato = 'dataframe'):
#Extract data from notes  
  corretora = definir_corretora(file_to_open, passwd)
  if (corretora == 'clear') :
    
    df_datas_c = extrair_data_clear(file_to_open, passwd, formato)
    df_negocios_c = extrair_negocios_clear(file_to_open,passwd, formato)
    df_custos_c = extrair_custos_clear(file_to_open,passwd, formato)
    #Take only the right bottom table (costs)
  
  #if (corretora == 'genial'):



  

  if formato == 'dataframe': return integrate(df_datas_c, df_negocios_c,df_custos_c)
  if formato == 'json': return (integrate(df_datas_c, df_negocios_c,df_custos_c))



In [63]:
#tests
files = ("/content/nota-de-corretagem (1).pdf", "/content/nota-de-corretagem.pdf", '/content/nota-de-corretagem (2).pdf')#, '/content/NotaCorretagem_30-06-20_30-06-20.pdf')
passwd = '007'

In [64]:
df_negocios = []
for file in files:
  df_negocio = extrair_dados(file, passwd, formato='json')
  df_negocios.append (df_negocio)

#df_negocios2, df_custos2 = extrair_dados(file2, passwd)

In [65]:

# json_return = ''
#for nota in df_negocios:
#  display(nota)
  #for pagina in nota:
#    for item in pagina:
      #display(pagina)
# #for custo in df_custos:
#   #display(custo)
#   #display(sum(custo['Valor']))
#json_return = json.dumps(df_negocios.to_json(orient='index'))
#json_return
#display(df_custos,df_negocios)
#df_negocios
display(df_negocios)
with open('/content/resultado.json','w',encoding='utf-8') as f:
  json.dump(df_negocios, f)

[[[[{'Data': '24/01/2022', 'Folha': 1, 'Nr. nota': 842471}],
   [{'C/V': 'V',
     'D/C': 'C',
     'Negociação': '1-BOVESPA',
     'Papel': 'ABEVB161ON 15,62',
     'Preço': 0.21,
     'Quantidade': 3900.0,
     'Tipo de mercado': 'OPCAO DE COMPRA',
     'Valor Operação': 819.0},
    {'C/V': 'V',
     'D/C': 'C',
     'Negociação': '1-BOVESPA',
     'Papel': 'ABEVB161ON 15,62',
     'Preço': 0.21,
     'Quantidade': 3900.0,
     'Tipo de mercado': 'OPCAO DE COMPRA',
     'Valor Operação': 819.0},
    {'C/V': 'V',
     'D/C': 'C',
     'Negociação': '1-BOVESPA',
     'Papel': 'ABEVB161ON 15,62',
     'Preço': 0.21,
     'Quantidade': 1600.0,
     'Tipo de mercado': 'OPCAO DE COMPRA',
     'Valor Operação': 336.0},
    {'C/V': 'V',
     'D/C': 'C',
     'Negociação': '1-BOVESPA',
     'Papel': 'ABEVB161ON 15,62',
     'Preço': 0.26,
     'Quantidade': 100.0,
     'Tipo de mercado': 'OPCAO DE COMPRA',
     'Valor Operação': 26.0}],
   [{'C/D': 'D', 'Custo': 'Taxa de liquidação', 'Valor':